# Direct Investment Current Cost Adjustments

This is an excerpt of a longer script that calculates (A) the current cost markup to the direct investment equity position and (B) the current cost adjustment to direct investment equity income.

(A) involves revaluing the following from historical cost to current cost:
1. structures & equipment (fixed assets)
2. inventories
3. land

(B) involves computing the consumption of fixed capital:
1. economic depreciation at current cost for structures & equipment (fixed assets)


For A.1, industry-by-year net fixed assets (obtained using entity-level direct investment data from BEA's DI surveys) are revalued to current cost from historical cost using ratios derived from BEA's Fixed Assets Accounts. For B.1, depreciation is computed using depreciation rates implied by the Fixed Assets Accounts. For both, since the necessary values are published, the BEA API can be used to retrieve them (rather than downloading excel files on our website).

To install *beaapi*, visit https://github.com/us-bea/beaapi/releases and download the wheel (.whl) file. Then, open your Anaconda terminal and run the following code (if a new version has been released, update the file name):
```
    pip install Downloads\beaapi-0.0.2-py3-none-any.whl
```

You will also need an API key. To register, visit https://apps.bea.gov/api/signup/ and fill out the registration form on the right hand side of the page. The API key will be emailed to you.

## I. Preliminaries

Enter your BEA API key:

In [1]:
beakey = '12345678-2222-3333-4444-000000000099'

Not all years of available data from the Fixed Assets Accounts are used for these calculations. They begin with 1995, and end with the most recent year of data available on MNEs from the annual DI surveys. Below, the desired start and end years are assigned to variables:

In [2]:
yearfirst = 1995
yearlast = 2020


Import packages needed for this code:

In [3]:
import numpy as np
import pandas as pd
import beaapi

## II. Import Fixed Assets Data

The *beaapi* ```get_data_set_list``` method returns the names and descriptions of the available table series:

In [4]:
beaapi.get_data_set_list(beakey)

DatasetName                                 DatasetDescription
0                      NIPA                               Standard NIPA tables
1        NIUnderlyingDetail               Standard NI underlying detail tables
2                       MNE                          Multinational Enterprises
3               FixedAssets                       Standard Fixed Assets tables
4                       ITA                International Transactions Accounts
5                       IIP                  International Investment Position
6               InputOutput                                  Input-Output Data
7             IntlServTrade                       International Services Trade
8               IntlServSTA  International Services Supplied Through Affili...
9             GDPbyIndustry                                    GDP by Industry
10                 Regional                                 Regional data sets
11  UnderlyingGDPbyIndustry                         Underlying GDP by Industry
12       APIDatasetMetaData                  Metadata about other API datasets

The ```get_parameter_list``` method returns the parameters for which we will need to pass arguments when retrieving the data:

In [5]:
beaapi.get_parameter_list(beakey, 'FixedAssets')

ParameterName ParameterDataType  \
0     TableName            string   
1          Year           integer   

                              ParameterDescription  ParameterIsRequiredFlag  \
0                  The new Fixed Assets identifier                        1   
1  List of year(s) of data to retrieve (X for All)                        1   

  ParameterDefaultValue  MultipleAcceptedFlag AllValue  
0                                           0           
1                                           1        X

The first parameter is **TableName**. Tables are obtained from BEA API using their table names (IDs), which can be found [here](https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf). Fixed Assets table names follow the naming convention ```FAAt###X```. I exploit this pattern using f-strings, and use abbreviated table names (just the numbers and letter) here:

In [6]:
# write python dict that maps (abbreviated) private FA table numbers to the descriptive col names that will be used later on to rename the "DataValue" col in each table:
pfatables = {'301E':'ccposequip',
             '301S':'ccposstruct',
             '303E':'hcposequip',
             '303S':'hcposstruct',
             '304E':'ccdeprequip',
             '304S':'ccdeprstruct'}
# same thing for residential FA tables:
rfatables = {'501':'ccposres',
             '503':'hcposres',
             '504':'ccdeprres'}

**Year** is the other parameter. Use ```Year='X'``` as the argument for all years, ```Year='YYYY'``` for a single year, or ```Year='YYYY, YYYY, YYYY, ...'``` for multiple. Notice that, for multiple years, the argument is not a list of strings, but rather a flat string with commas between years. 

For many years it may be tedious to type them all manually, so you may wish to convert a list of numeric years to the flat string:

In [7]:
# create a list of numeric years using the variables defined in the previous section:
yearlist = list(np.arange(yearfirst,yearlast,1))
# create an empty string:
yearstr = ""
# for each item in the list of years (up until the last one), add that year (as a string) plus a comma with whitespace character, to the empty string:
for i in range(len(yearlist)):
    yearstr += str(yearlist[i]) + ", "
# for the last year, just add the year as a string, without the comma and whitespace character:
yearstr += str(yearlast)

print(type(yearlist), yearlist) # list of numeric years
print(type(yearstr), yearstr) # flat string of years

<class 'list'> [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
<class 'str'> 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020


To retrieve data using the BEA API, use the ```get_data``` method. For the Fixed Assets tables, where the parameters are **TableName** and **Year**, with **TableName** taking the form ```FAAt###X```, the code is as follows:
```
beaapi.get_data(beakey, 'FixedAssets', TableName=f'FAAt{###X}', Year='YYYY')
```

In [8]:
# iteratively import private fixed assets by industry tables (and save each to a pre-defined empty list of dataframes):
fatables = []
for i in list(pfatables.keys()):
    # all fixed assets tables follow the same naming convention FAAt###X, so I use an f-string with abbreviated table names (keys) from the dicts in the cell above:
    fatable = beaapi.get_data(beakey, 'FixedAssets', TableName=f'FAAt{i}', Year=yearstr)[['LineNumber', 'LineDescription', 'TimePeriod', 'DataValue']] # selects only the cols needed for this code
    fatables.append(fatable)

# same thing for residential fixed assets tables (append to same list of dataframes):
for j in list(rfatables.keys()):
    fatable = beaapi.get_data(beakey, 'FixedAssets', TableName=f'FAAt{j}', Year=yearstr)[['LineNumber', 'LineDescription', 'TimePeriod', 'DataValue']]
    # only want the private aggregate lines for these tables:
    fatable.drop(fatable.loc[fatable['LineDescription']!='Private'].index, axis=0, inplace=True)
    # replace "LineNumber" values with 1 for consistency with private fixed assets tables:
    fatable.replace({'LineNumber':{2:1}}, inplace=True)
    fatables.append(fatable)

# can view each dataframe in the list of dfs using list indexing:
fatables[0]

LineNumber                    LineDescription TimePeriod  DataValue
0              1               Private fixed assets       1995    2994776
1              1               Private fixed assets       1996    3140588
2              1               Private fixed assets       1997    3271617
3              1               Private fixed assets       1998    3421541
4              1               Private fixed assets       1999    3616329
...          ...                                ...        ...        ...
2491          96  Other services, except government       2016      99856
2492          96  Other services, except government       2017      99745
2493          96  Other services, except government       2018     101753
2494          96  Other services, except government       2019     103430
2495          96  Other services, except government       2020     105439

[2496 rows x 4 columns]

Each table has a **DataValue** column, though the value represented in it is different in each table. Using the column names (values) from the two dictionaries, these can be renamed for clarity:

In [9]:
# create a combined list of column names (values) from the private and residental fixed assets dicts:
values = list(pfatables.values()) + list(rfatables.values())

# rename the "DataValue" column in each dataframe with the appropriate, more descriptive column name: 
for i in values:
    fatable = values.index(i) # gets the index position of the appropriate column name in the list of all column names
    fatables[fatable].rename({'DataValue':i}, axis=1, inplace=True) # rename the "DataValue" column in the relevant dataframe

fatables[0].head()

LineNumber       LineDescription TimePeriod  ccposequip
0           1  Private fixed assets       1995     2994776
1           1  Private fixed assets       1996     3140588
2           1  Private fixed assets       1997     3271617
3           1  Private fixed assets       1998     3421541
4           1  Private fixed assets       1999     3616329

Now that there are unique column names for the data values in each table, they can be merged:

In [10]:
# merge private fixed assets tables:
fa = fatables[0]
for i in range(1,len(pfatables)):
    fa = pd.merge(fa, fatables[i], on=['LineNumber', 'TimePeriod', 'LineDescription'])

print('fa', fa.shape)
# add residential fixed assets:
for j in range(len(pfatables), len(pfatables) + len(rfatables)):
    fa = pd.merge(fa, fatables[j].drop(['LineDescription'], axis=1), on=['LineNumber', 'TimePeriod'], how='left')

# residential fixed assets data were obtained only for the "Private" aggregate line; fill null values for the other lines with zero:
fa.fillna(0, inplace=True)

fa.head(), fa.tail()

fa (2496, 9)


(   LineNumber       LineDescription TimePeriod  ccposequip  ccposstruct  \
 0           1  Private fixed assets       1995     2994776     12105745   
 1           1  Private fixed assets       1996     3140588     12714845   
 2           1  Private fixed assets       1997     3271617     13438018   
 3           1  Private fixed assets       1998     3421541     14271166   
 4           1  Private fixed assets       1999     3616329     15203793   
 
    hcposequip  hcposstruct  ccdeprequip  ccdeprstruct   ccposres   hcposres  \
 0     2535898      6454856       398516        295031  7227953.0  3723818.0   
 1     2720384      6841577       419640        308620  7615419.0  3968634.0   
 2     2917240      7258084       440883        326273  8028971.0  4223139.0   
 3     3129521      7727779       461388        348201  8548636.0  4511733.0   
 4     3358112      8231945       488501        369742  9177649.0  4832294.0   
 
    ccdeprres  
 0   167364.0  
 1   175615.0  
 2   184802.

## III. Perform Calculations

First, calculate the current cost (CC) position to historical cost (HC) position ratios (overall, nonresidential, and residential) for each line and year:

In [11]:
fa['cchcratio_fa'] = (fa['ccposequip'] + fa['ccposstruct']) / (fa['hcposequip'] + fa['hcposstruct'])
fa['cchcratio_fa_nonres'] = (fa['ccposequip'] + fa['ccposstruct'] - fa['ccposres']) / (fa['hcposequip'] + fa['hcposstruct'] - fa['hcposres'])
fa['cchcratio_fa_res'] = fa['ccposres'] / fa['hcposres']

fa.head(), fa.tail()


(   LineNumber       LineDescription TimePeriod  ccposequip  ccposstruct  \
 0           1  Private fixed assets       1995     2994776     12105745   
 1           1  Private fixed assets       1996     3140588     12714845   
 2           1  Private fixed assets       1997     3271617     13438018   
 3           1  Private fixed assets       1998     3421541     14271166   
 4           1  Private fixed assets       1999     3616329     15203793   
 
    hcposequip  hcposstruct  ccdeprequip  ccdeprstruct   ccposres   hcposres  \
 0     2535898      6454856       398516        295031  7227953.0  3723818.0   
 1     2720384      6841577       419640        308620  7615419.0  3968634.0   
 2     2917240      7258084       440883        326273  8028971.0  4223139.0   
 3     3129521      7727779       461388        348201  8548636.0  4511733.0   
 4     3358112      8231945       488501        369742  9177649.0  4832294.0   
 
    ccdeprres  cchcratio_fa  cchcratio_fa_nonres  cchcratio_

Next, calculate the depreciation rate (overall, nonresidential, and residential) for each line and year:

In [12]:
# the depreciation rate calculate requires the lag of the CC positions (equipment, structure, and residential) for each line and year:
fa['ccposequip_lag'] = fa.sort_values(['LineNumber', 'TimePeriod']).groupby('LineNumber')['ccposequip'].shift(1)
fa['ccposstruct_lag'] = fa.sort_values(['LineNumber', 'TimePeriod']).groupby('LineNumber')['ccposstruct'].shift(1)
fa['ccposres_lag'] = fa.sort_values(['LineNumber', 'TimePeriod']).groupby('LineNumber')['ccposres'].shift(1)

fa.head()

LineNumber       LineDescription TimePeriod  ccposequip  ccposstruct  \
0           1  Private fixed assets       1995     2994776     12105745   
1           1  Private fixed assets       1996     3140588     12714845   
2           1  Private fixed assets       1997     3271617     13438018   
3           1  Private fixed assets       1998     3421541     14271166   
4           1  Private fixed assets       1999     3616329     15203793   

   hcposequip  hcposstruct  ccdeprequip  ccdeprstruct   ccposres   hcposres  \
0     2535898      6454856       398516        295031  7227953.0  3723818.0   
1     2720384      6841577       419640        308620  7615419.0  3968634.0   
2     2917240      7258084       440883        326273  8028971.0  4223139.0   
3     3129521      7727779       461388        348201  8548636.0  4511733.0   
4     3358112      8231945       488501        369742  9177649.0  4832294.0   

   ccdeprres  cchcratio_fa  cchcratio_fa_nonres  cchcratio_fa_res  \
0   167364.0      1.679561             1.494715          1.941006   
1   175615.0      1.658178             1.473187          1.918902   
2   184802.0      1.642172             1.458400          1.901186   
3   196474.0      1.629568             1.441017          1.894757   
4   211460.0      1.623816             1.426874          1.899232   

   ccposequip_lag  ccposstruct_lag  ccposres_lag  
0             NaN              NaN           NaN  
1       2994776.0       12105745.0     7227953.0  
2       3140588.0       12714845.0     7615419.0  
3       3271617.0       13438018.0     8028971.0  
4       3421541.0       14271166.0     8548636.0

In [13]:
fa['deprrate_fa'] = (fa['ccdeprequip'] + fa['ccdeprstruct']) / (0.5 * ((fa['ccposequip'] + fa['ccposstruct']) + 
                                                                       (fa['ccposequip_lag'] + fa['ccposstruct_lag'])))
fa['deprrate_fa_nonres'] = (fa['ccdeprequip'] + fa['ccdeprstruct'] - fa['ccdeprres']) / (0.5 * ((fa['ccposequip'] + fa['ccposstruct'] - fa['ccposres']) + 
                                                                                                (fa['ccposequip_lag'] + fa['ccposstruct_lag'] - fa['ccposres_lag'])))
fa['deprrate_fa_res'] = fa['ccdeprres'] / (0.5 * (fa['ccposres'] + 
                                                  fa['ccposres_lag']))

fa.head()

LineNumber       LineDescription TimePeriod  ccposequip  ccposstruct  \
0           1  Private fixed assets       1995     2994776     12105745   
1           1  Private fixed assets       1996     3140588     12714845   
2           1  Private fixed assets       1997     3271617     13438018   
3           1  Private fixed assets       1998     3421541     14271166   
4           1  Private fixed assets       1999     3616329     15203793   

   hcposequip  hcposstruct  ccdeprequip  ccdeprstruct   ccposres  ...  \
0     2535898      6454856       398516        295031  7227953.0  ...   
1     2720384      6841577       419640        308620  7615419.0  ...   
2     2917240      7258084       440883        326273  8028971.0  ...   
3     3129521      7727779       461388        348201  8548636.0  ...   
4     3358112      8231945       488501        369742  9177649.0  ...   

   ccdeprres  cchcratio_fa  cchcratio_fa_nonres  cchcratio_fa_res  \
0   167364.0      1.679561             1.494715          1.941006   
1   175615.0      1.658178             1.473187          1.918902   
2   184802.0      1.642172             1.458400          1.901186   
3   196474.0      1.629568             1.441017          1.894757   
4   211460.0      1.623816             1.426874          1.899232   

   ccposequip_lag  ccposstruct_lag  ccposres_lag  deprrate_fa  \
0             NaN              NaN           NaN          NaN   
1       2994776.0       12105745.0     7227953.0     0.047051   
2       3140588.0       12714845.0     7615419.0     0.047115   
3       3271617.0       13438018.0     8028971.0     0.047066   
4       3421541.0       14271166.0     8548636.0     0.047010   

   deprrate_fa_nonres  deprrate_fa_res  
0                 NaN              NaN  
1            0.068598         0.023662  
2            0.068833         0.023625  
3            0.068794         0.023704  
4            0.068856         0.023858  

[5 rows x 21 columns]

Finally, only the calculated CC/HC ratios and depreciation rate columns for each line and year are used later on, so the remaining columns can be dropped:

In [14]:
fa.drop(['ccposequip', 'ccposstruct', 'hcposequip', 'hcposstruct', 'ccdeprequip', 'ccdeprstruct', 
         'ccposres', 'hcposres', 'ccdeprres',
         'ccposequip_lag', 'ccposstruct_lag', 'ccposres_lag'], axis=1, inplace=True)

fa

LineNumber                    LineDescription TimePeriod  cchcratio_fa  \
0              1               Private fixed assets       1995      1.679561   
1              1               Private fixed assets       1996      1.658178   
2              1               Private fixed assets       1997      1.642172   
3              1               Private fixed assets       1998      1.629568   
4              1               Private fixed assets       1999      1.623816   
...          ...                                ...        ...           ...   
2491          96  Other services, except government       2016      1.787142   
2492          96  Other services, except government       2017      1.808258   
2493          96  Other services, except government       2018      1.859758   
2494          96  Other services, except government       2019      1.884819   
2495          96  Other services, except government       2020      1.899064   

      cchcratio_fa_nonres  cchcratio_fa_res  deprrate_fa  deprrate_fa_nonres  \
0                1.494715          1.941006          NaN                 NaN   
1                1.473187          1.918902     0.047051            0.068598   
2                1.458400          1.901186     0.047115            0.068833   
3                1.441017          1.894757     0.047066            0.068794   
4                1.426874          1.899232     0.047010            0.068856   
...                   ...               ...          ...                 ...   
2491             1.787142               NaN     0.043699            0.043699   
2492             1.808258               NaN     0.043581            0.043581   
2493             1.859758               NaN     0.043293            0.043293   
2494             1.884819               NaN     0.042991            0.042991   
2495             1.899064               NaN     0.042822            0.042822   

      deprrate_fa_res  
0                 NaN  
1            0.023662  
2            0.023625  
3            0.023704  
4            0.023858  
...               ...  
2491              NaN  
2492              NaN  
2493              NaN  
2494              NaN  
2495              NaN  

[2496 rows x 9 columns]

The CC/HC ratios are used later to revalue industry-level net fixed assets from historical cost to current cost (an element in the calculation of the CC markup to DI equity position), and the depreciation rate to compute industry-level economic depreciation at current cost (an element in the calculation of the CC adjustment to DI equity income).